# Generate `dim_mob_researchers` table

## Definitions

In [1]:
import pandas as pd
import time

from tqdm import tqdm
from dotenv import load_dotenv
load_dotenv('../.env')
from sqlalchemy import create_engine
engine = create_engine(
    os.getenv('DB_CONNECTION_STRING'))

DIMENSIONS_DATABASE = os.getenv('DIMENSIONS_DATABASE') ### Not disclosed due security
USER_DATABASE = os.getenv('USER_DATABASE') ### Not disclosed due security

## list institutions by number of 2018 researchers

In [6]:
affils = pd.read_sql_query('''
select 
    r.*,
    o.organization_name,
    o.country_code
from (
    select
        dmp.grid_id,
        count(distinct researcher_id) as researchers
    from 
        dim_mob_pub dmp
    where 
        researcher_id is not null
        and 
        pub_year = 2018
    group by 
        dmp.grid_id
) as r
inner join {DIMENSIONS_DATABASE}.dbo.organization o on r.grid_id = o.grid_id
where researchers >= 100
order by r.researchers desc
'''.format(DIMENSIONS_DATABASE=DIMENSIONS_DATABASE),engine)
affils

,grid_id,researchers,organization_name,country_code
0,grid.11899.38,17216,University of Sao Paulo,BR
1,grid.410726.6,16804,University of Chinese Academy of Sciences,CN
2,grid.38142.3c,14070,Harvard University,US
3,grid.4444.0,13592,French National Centre for Scientific Research,FR
4,grid.13402.34,13419,Zhejiang University,CN
...,...,...,...,...
5897,grid.498067.4,100,Centre for Research and Teaching in Environmen...,FR
5898,grid.444459.c,100,Abu Dhabi University,AE
5899,grid.452651.1,100,National Institute of Genomic Medicine,MX
5900,grid.454303.5,100,National Chin-Yi University of Technology,TW


## List all disciplines

In [7]:
disciplines = pd.read_sql_query(f'select * from {DIMENSIONS_DATABASE}.dbo.for_division',engine,index_col='for_division_id')

In [1]:
def getResearcherMobility(df,grid_id,country_code,T_treshold=2):
    beg = df[(df['T'] < T_treshold)]
    pub_gb = df.groupby('pub_id')
    orgs_per_paper = pub_gb.apply(lambda g:g.grid_id.unique.shape[0])
    same_org = pub_gb.apply(lambda g: (g.grid_id == grid_id).any())
    other_org = pub_gb.apply(lambda g: (g.grid_id != grid_id).any())
    same_country = pub_gb.apply(lambda g: (g.country_code == country_code).any())
    other_country = pub_gb.apply(lambda g: (g.country_code != country_code).any())
    T = df['T'].max()

    #df_sorted = df.sort_values(by='pub_year')
    return pd.Series({
        'N': df.N.max(),
        'T': T,
        'sameInst': (beg.grid_id == grid_id).any(),
        'sameCountry': (beg.country_code == country_code).any(),
        'shareSameInst':df[df.grid_id == grid_id].shape[0]/df.shape[0],
        'shareSameCountry':df[df.country_code == country_code].shape[0]/df.shape[0],
        'Orgs':df.grid_id.unique().shape[0],
        'Orgs_2018':df[df.pub_year == 2018].grid_id.unique(),
        'AvgOrgsPerPaper': orgs_per_paper.mean(),
        'MedOrgsPerPaper':orgs_per_paper.median(),
    })

dfs = []
for row in tqdm(list(affils.itertuples())):
    for disc in disciplines.index.unique():
        start = time.time()
        pub = pd.read_sql_query("""
        select 
            * 
        from 
            dim_mob_pub
        where 
            researcher_id in (
                select distinct 
                researcher_id
            from 
                dim_mob_pub
            where
                researcher_id is not null
                and 
                pub_year = 2018
                and 
                grid_id = '{grid}'
                and
                for_division_id = '{discipline}'
            )
        """.format(**{'grid':row.grid_id,'discipline':disc}),engine)
        gb = pub.groupby('researcher_id')

        rmob = gb.apply(lambda g: getResearcherMobility(g,row.grid_id,row.country_code))
        rmob['grid_id'] = row.grid_id
        #rmob['org_name'] = row.organization_name
        rmob['country_code'] = row.country_code
        rmob['for_division_id'] = disc#disciplines.loc[disc].for_division
        rmob.reset_index().to_sql('dim_mob_researchers',engine,if_exists='append',index=False)
        end = time.time()
        #print('{}, {} parsed in {}s'.format(row.organization_name,disciplines.loc[disc].for_division,end-start))

NameError: name 'tqdm' is not defined